# T1 - Reprise du modèle défini à la partie 1

Notre modèle dimensionnel révisé comprend :
- 2 tables de faits : FACT_SALES et FACT_INVENTORY
- 5 dimensions : DIM_TIME, DIM_PRODUCT, DIM_CUSTOMER, DIM_GEOGRAPHY, DIM_DEPARTMENT

Vérification de la faisabilité avec les données disponibles :
- Pour FACT_SALES : Nous avons orders.csv et order_details.csv qui contiennent les données nécessaires
- Pour FACT_INVENTORY : Nous devrons dériver cette table à partir des ventes et des capacités de stockage
- Pour les dimensions : Nous avons les fichiers correspondants (customers.csv, products.csv, etc.)

Le modèle est donc réalisable avec les données disponibles, bien que certaines mesures devront être calculées ou estimées.

# T2 - Importation des fichiers CSV et inspection des structures

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re

In [4]:
# Importation des fichiers CSV
addresses_df = pd.read_csv('data/addresses.csv')
customers_df = pd.read_csv('data/customers.csv')
departments_df = pd.read_csv('data/departments.csv')
order_details_df = pd.read_csv('data/order_details.csv')
orders_df = pd.read_csv('data/orders.csv')
products_df = pd.read_csv('data/products.csv')

In [5]:
# Inspection des structures
print("=== Aperçu de addresses_df ===")
print(addresses_df.info())
print(addresses_df.head())
print("\n=== Aperçu de customers_df ===")
print(customers_df.info())
print(customers_df.head())
print("\n=== Aperçu de departments_df ===")
print(departments_df.info())
print(departments_df.head())
print("\n=== Aperçu de order_details_df ===")
print(order_details_df.info())
print(order_details_df.head())
print("\n=== Aperçu de orders_df ===")
print(orders_df.info())
print(orders_df.head())
print("\n=== Aperçu de products_df ===")
print(products_df.info())
print(products_df.head())

=== Aperçu de addresses_df ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4910 entries, 0 to 4909
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Address ID   4910 non-null   object
 1   Customer ID  4910 non-null   object
 2   Country      4910 non-null   object
 3   City         4910 non-null   object
 4   State        4910 non-null   object
 5   Postal Code  4910 non-null   int64 
 6   Region       4910 non-null   object
dtypes: int64(1), object(6)
memory usage: 268.6+ KB
None
       Address ID Customer ID        Country             City           State  \
0  CG-12520_42420    CG-12520  United States        Henderson        Kentucky   
1  DV-13045_90036    DV-13045  United States      Los Angeles      California   
2  SO-20335_33311    SO-20335  United States  Fort Lauderdale         Florida   
3  BH-11710_90032    BH-11710  United States      Los Angeles      California   
4  AA-10480_28027    AA-10480  U

In [6]:
# Vérification des valeurs manquantes
print("\n=== Valeurs manquantes ===")
for df_name, df in [("addresses_df", addresses_df), 
                    ("customers_df", customers_df),
                    ("departments_df", departments_df),
                    ("order_details_df", order_details_df),
                    ("orders_df", orders_df),
                    ("products_df", products_df)]:
    missing_values = df.isnull().sum()
    if missing_values.sum() > 0:
        print(f"{df_name} contient des valeurs manquantes:")
        print(missing_values[missing_values > 0])
    else:
        print(f"{df_name} ne contient pas de valeurs manquantes")


=== Valeurs manquantes ===
addresses_df ne contient pas de valeurs manquantes
customers_df ne contient pas de valeurs manquantes
departments_df ne contient pas de valeurs manquantes
order_details_df ne contient pas de valeurs manquantes
orders_df ne contient pas de valeurs manquantes
products_df ne contient pas de valeurs manquantes


In [7]:
# Vérification des doublons
print("\n=== Doublons ===")
for df_name, df in [("addresses_df", addresses_df), 
                    ("customers_df", customers_df),
                    ("departments_df", departments_df),
                    ("order_details_df", order_details_df),
                    ("orders_df", orders_df),
                    ("products_df", products_df)]:
    duplicates = df.duplicated().sum()
    print(f"{df_name} contient {duplicates} doublons")

# Vérification des types de données
print("\n=== Types de données ===")
for df_name, df in [("addresses_df", addresses_df), 
                    ("customers_df", customers_df),
                    ("departments_df", departments_df),
                    ("order_details_df", order_details_df),
                    ("orders_df", orders_df),
                    ("products_df", products_df)]:
    print(f"{df_name} types:")
    print(df.dtypes)


=== Doublons ===
addresses_df contient 0 doublons
customers_df contient 0 doublons
departments_df contient 0 doublons
order_details_df contient 0 doublons
orders_df contient 0 doublons
products_df contient 0 doublons

=== Types de données ===
addresses_df types:
Address ID     object
Customer ID    object
Country        object
City           object
State          object
Postal Code     int64
Region         object
dtype: object
customers_df types:
Customer ID      object
Customer Name    object
Segment          object
dtype: object
departments_df types:
Department          object
City                object
Storage Capacity     int64
dtype: object
order_details_df types:
Order ID       object
Product ID     object
Sales         float64
Quantity        int64
Profit        float64
dtype: object
orders_df types:
Order ID       object
Order Date     object
Ship Date      object
Customer ID    object
Address ID     object
dtype: object
products_df types:
Product ID      object
Department    